# GSOD

## Select Station

选取模型中需要的站点

In [ ]:
import pandas as pd

# 以下两个文件中存储的站点可根据自己的需求定义，示例代码提供的为2001年1月1日至2019年3月26日0-60°N，65°E-145°E
# 文件缺失和数据大量（缺失天数大于10%）缺失的站点

# 生成这两个文件的代码我找不到了，但我相信这你们来说并不难(ง •_•)ง

unuse_nofile = pd.read_csv('DATA\\unuse_nofile.csv', low_memory=False).get_values()  # unuse_nofile.csv 存储的是文件缺失的站点

unuse_nofile = [row[0] for row in unuse_nofile]

unuse_lossdata = pd.read_csv('DATA\\unuse_lossdata.csv', low_memory=False).get_values()  # unuse_lossdata.csv 存储的是数据大量缺失的站点

unuse_lossdata = [row[0] for row in unuse_lossdata]

unuse = unuse_nofile + unuse_lossdata

data = pd.read_csv('DATA\\isd-history.csv', low_memory=False)

stats = []

i = 0

for row in data.get_values():

    if row[8] < 20010101 and row[9] > 20190325:

        if row[5] > 0 and row[5] < 60 and row[6] > 65 and row[6] < 145:
            
            if row[0].zfill(6) + '-' + str(row[1]).zfill(5) not in unuse:

                stats += [[row[0].zfill(6) + '-' + str(row[1]).zfill(5), row[2], row[3], row[4], row[5], row[6], row[7]]]

stats = pd.DataFrame(stats, columns=['USAF-WBAN', 'STATION NAME', 'CTRY', 'ST CALL', 'LAT', 'LON', 'ELEV(m)'])

stats.to_csv('DATA\\stats.csv', index = False)

![title](IMG/use_stats.png)

In [ ]:
import pandas as pd

stats = pd.read_csv('DATA\\stats.csv', low_memory=False)

from shutil import copyfile

for year in range(2001, 2020):

    for usaf_wban in stats['USAF-WBAN'].get_values():

        try:

            copyfile('GSOD\\gsod_' + str(year) + '\\' + usaf_wban + '-' + str(year) + '.op.gz', 'GSOD\\new_data\\' + usaf_wban + '-' + str(year) + '.op.gz')
            # 把挑选的文件转移到一个新的文件夹（这一步其实是多余的）
            # GSOD文件在网盘，没有传到Github上来
        except Exception:

            print(usaf_wban, year)  # 如果有报错说明该文件缺失

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os

date0 = datetime.datetime.strptime('20010101', "%Y%m%d")

def line2desc(data):

    stn = data[:6]
    wban = data[7:12]
    date = (datetime.datetime.strptime(data[14:22], "%Y%m%d") - date0).days
    
    temp = float(data[24:30])
    if temp == 9999.9:
        temp = np.nan
        
    dewp = float(data[35:41])
    if dewp == 9999.9:
        dewp = np.nan

    slp = float(data[46:52])
    if slp == 9999.9:
        slp = np.nan

    stp = float(data[57:63])
    if stp == 9999.9:
        stp = np.nan

    MAX = float(data[102:108])
    if MAX == 9999.9:
        MAX = np.nan

    MIN = float(data[110:116])
    if MIN == 9999.9:
        MIN = np.nan

    prcp = float(data[118:123])
    if prcp == 99.99:
        prcp = np.nan

    return [stn + '-' + wban, date, temp, dewp, slp, stp, MAX, MIN, prcp]

folder = 'GSOD\\new_data'

files = os.listdir(folder)

total = np.zeros((6574, 5579)) + np.nan

n = 0

for file in files:

    j = n // 18  # 两个站点之间相隔18个文件，19年的数据不加入

    year = int(file[13:17])

    if year == 2019:

        continue

    data_temp = pd.read_csv(folder + '\\' + file, low_memory=False).get_values()  # 读取每个站点每年的数据

    for row in data_temp:

        desc = line2desc(row[0])  # 将一行数据转化为需要的数据

        line = np.array(desc[2:])

        total[desc[1], j * 7:j * 7 + 7] = line  # 每7列一个站点，desc[1]是从2001年1月1日至数据记录时的天数，也就是在矩阵中的行数

    n += 1

use_col = []

for j in range(5579):

    if np.sum(np.isnan(total[:, j])) < 657:  # 选取缺失数据小于10%的变量

        use_col += [j]

total = total[:, use_col]

np.save('total_used.npy', total)